# Retriever
- 비정형 질의(query)를 입력 받아 Vector store에서 관련된 내용을 검색하는 기능을 제공하는 인터페이스로 다양한 데이터 소스에서 정보를 검색하여 대규모 언어 모델(LLM) 기반 애플리케이션의 **정확성을** 향상시키는 데 핵심적인 역할을 한다.

![RAG](figures/rag2.png)

## 주요 특징
- **다양한 데이터 소스 지원**
	- Retriever는 벡터 스토어, 그래프 데이터베이스, 관계형 데이터베이스 등 여러 종류의 검색 시스템과 상호작용할 수 있는 통일된 인터페이스를 제공.
- **간단한 인터페이스**: Retriever는 문자열 형태의 쿼리를 입력받아 관련 문서의 리스트를 반환한다. 이러한 단순한 구조 덕분에 다양한 검색 시스템과 쉽게 통합할 수 있다. 

## 다양한 Retriever 유형
   - **벡터 스토어(Vector Store) Retriever**
		- 텍스트 조각마다 임베딩을 생성하여 유사도 검색을 수행합니다. 빠르고 간단한 검색 시스템을 구축할 때 적합하다.
   - **ParentDocument Retriever**
		- 문서를 여러 청크로 나누어 인덱싱한 후, 가장 유사한 청크를 찾아 전체 원본 문서를 반환한다. 작은 정보 조각들이 모여 하나의 문서를 구성할 때 유용하다.
   - **멀티 벡터(Multi Vector) Retriever**
		- 각 문서에 대해 요약이나 가상의 질문 생성과 수동 추가 방식을 통해 문서당 여러 벡터를 생성하여 인덱싱한다. 텍스트 자체보다 더 관련성 있는 정보를 추출할 수 있을 때 사용한다.
   - **Self Query Retriever**
		- LLM을 활용하여 사용자 입력을 텍스트 검색어와 메타데이터 필터로 변환한다. 문서의 메타데이터에 대한 질문을 처리할 때 효과적.
   - **Contextual Compression Retriever**
		- 검색된 문서에서 불필요한 정보를 제거하고, 쿼리와 관련된 핵심 내용만을 추출한다. 

## **고급 검색 패턴 지원**:
   - **앙상블(Ensemble) Retriever**: 여러 Retriever의 검색 결과를 결합하여 더 정확한 결과를 제공한다.
   - **소스 문서 보존(Source Document Retention)**: 인덱싱 과정에서 변환된 문서와 원본 문서 간의 연결을 유지하여, 검색 시 원본 문서를 반환할 수 있게 한다. 

> **인덱싱**
> - 인덱싱은 벡터화된 문서들을 효율적으로 저장하고 관리하여, 유사성 검색 시 빠르게 원하는 정보를 찾을 수 있도록 하는 과정을 말한다. 이를 통해 벡터화된 데이터의 신속하게 찾을 수있다.

In [2]:
from langchain_community.document_loaders import TextLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_openai import OpenAIEmbeddings
from langchain_ollama import OllamaEmbeddings
from langchain_chroma import Chroma
from dotenv import load_dotenv

load_dotenv()

True

In [ ]:
# COLLECTION_NAME = "olympic_db" # ollama embedding 
COLLECTION_NAME = "olympic_db_openai"
PERSIST_DIRECTORY = "vector_store/chroma/olympic_db"

EMBEDDING_MODEL_NAME = "text-embedding-3-small"
embedding_model = OpenAIEmbeddings(model=EMBEDDING_MODEL_NAME)

# EMBEDDING_MODEL_NAME = "nomic-embed-text"
# embedding_model = OllamaEmbeddings(model=EMBEDDING_MODEL_NAME)

# vector store 연결
vector_store = Chroma(
    embedding_function=embedding_model,
    collection_name=COLLECTION_NAME,
    persist_directory=PERSIST_DIRECTORY
)

In [4]:
# Retriever(Runnable 타입 -> LCEL의 체인에 들어갈 수있다.)를 생성 - 조회대상 vector_store로 부터 생성
## query와 유사도가 높은 문서들을 조회하는 기능을 제공.
retriever = vector_store.as_retriever()

# vector store에서 query와 관련된 문서 조회.
query = "국제 올림픽 위원회에 대해 알려주세요."
result = retriever.invoke(query) # 문자열로 query를 전달. (dictionary사용하지 않음.)

In [5]:
result

[Document(metadata={'source': 'data/olympic.txt'}, page_content='국제 올림픽 위원회\n올림픽 활동이란 많은 수의 국가, 국제 경기 연맹과 협회 • 미디어 파트너를 맺기 • 선수, 직원, 심판, 모든 사람과 기관이 올림픽 헌장을 지키는 것을 말한다. 국제올림픽위원회(IOC)는 모든 올림픽 활동을 통솔하는 단체로서, 올림픽 개최 도시 선정, 계획 감독, 종목 변경, 스폰서 및 방송권 계약 체결 등의 권리가 있다. 올림픽 활동은 크게 세 가지로 구성된다.'),
 Document(metadata={'source': 'data/olympic.txt'}, page_content='- 국가 올림픽 위원회(NOC)는 각국의 올림픽 활동을 감독하는 기구이다. 예를 들어서 대한 올림픽 위원회(KOC)는 대한민국의 국가 올림픽 위원회이다. 현재 IOC에 소속된 국가 올림픽 위원회는 205개이다.\n- 올림픽 조직 위원회(OCOG)는 임시적인 조직으로 올림픽의 총체적인 것(개막식, 페막식 등)을 책임지기 위해 구성된 조직이다. 올림픽 조직 위원회는 올림픽이 끝나면 해산되며 최종보고서를 IOC에 제출한다.'),
 Document(metadata={'source': 'data/olympic.txt'}, page_content='올림픽은 국제경기연맹(IF), 국가 올림픽 위원회(NOC), 각 올림픽의 위원회(예-벤쿠버동계올림픽조직위원회)로 구성된다. 의사 결정 기구인 IOC는 올림픽 개최 도시를 선정하며, 각 올림픽 대회마다 열리는 올림픽 종목도 IOC에서 결정한다. 올림픽 경기 개최 도시는 경기 축하 의식이 올림픽 헌장에 부합하도록 조직하고 기금을 마련해야 한다. 올림픽 축하 행사로는 여러 의식과 상징을 들 수 있는데 올림픽기나 성화가 그 예이다.'),
 Document(metadata={'source': 'data/olympic.txt'}, page_content='설립했다. 여기서는 올림픽 종목과 올림픽 종목이 아닌 스포츠를 모두

In [ ]:
# 조회 조건, 설정 들을 as_retriever() search_kwargs={"설정명":"설정값", ..} 인자로 전달.
retriever2 = vector_store.as_retriever(
    search_kwargs = {
        "k":2,
        "filter":{"source":"data/olympic.txt"}
    }
)
retriever2.invoke(query)

# "filter":{"year":{"$ge":2020}}  # metadata의 year >= 2020

[Document(metadata={'source': 'data/olympic.txt'}, page_content='국제 올림픽 위원회\n올림픽 활동이란 많은 수의 국가, 국제 경기 연맹과 협회 • 미디어 파트너를 맺기 • 선수, 직원, 심판, 모든 사람과 기관이 올림픽 헌장을 지키는 것을 말한다. 국제올림픽위원회(IOC)는 모든 올림픽 활동을 통솔하는 단체로서, 올림픽 개최 도시 선정, 계획 감독, 종목 변경, 스폰서 및 방송권 계약 체결 등의 권리가 있다. 올림픽 활동은 크게 세 가지로 구성된다.'),
 Document(metadata={'source': 'data/olympic.txt'}, page_content='- 국가 올림픽 위원회(NOC)는 각국의 올림픽 활동을 감독하는 기구이다. 예를 들어서 대한 올림픽 위원회(KOC)는 대한민국의 국가 올림픽 위원회이다. 현재 IOC에 소속된 국가 올림픽 위원회는 205개이다.\n- 올림픽 조직 위원회(OCOG)는 임시적인 조직으로 올림픽의 총체적인 것(개막식, 페막식 등)을 책임지기 위해 구성된 조직이다. 올림픽 조직 위원회는 올림픽이 끝나면 해산되며 최종보고서를 IOC에 제출한다.')]

In [9]:
### MMR 방식 조회
retriever3 = vector_store.as_retriever(
    search_type="mmr",  #MMR 방식을 지정.
    search_kwargs = {
        "k":5,
        "fetch_k": 10,
        "lambda_mult":0.2, # 1<- 유사도를 강조, 0<- 다양성을 강조
        "filter":{"source":"data/olympic.txt"}
    }
)
retriever3.invoke(query)

[Document(metadata={'source': 'data/olympic.txt'}, page_content='국제 올림픽 위원회\n올림픽 활동이란 많은 수의 국가, 국제 경기 연맹과 협회 • 미디어 파트너를 맺기 • 선수, 직원, 심판, 모든 사람과 기관이 올림픽 헌장을 지키는 것을 말한다. 국제올림픽위원회(IOC)는 모든 올림픽 활동을 통솔하는 단체로서, 올림픽 개최 도시 선정, 계획 감독, 종목 변경, 스폰서 및 방송권 계약 체결 등의 권리가 있다. 올림픽 활동은 크게 세 가지로 구성된다.'),
 Document(metadata={'source': 'data/olympic.txt'}, page_content='- 국가 올림픽 위원회(NOC)는 각국의 올림픽 활동을 감독하는 기구이다. 예를 들어서 대한 올림픽 위원회(KOC)는 대한민국의 국가 올림픽 위원회이다. 현재 IOC에 소속된 국가 올림픽 위원회는 205개이다.\n- 올림픽 조직 위원회(OCOG)는 임시적인 조직으로 올림픽의 총체적인 것(개막식, 페막식 등)을 책임지기 위해 구성된 조직이다. 올림픽 조직 위원회는 올림픽이 끝나면 해산되며 최종보고서를 IOC에 제출한다.'),
 Document(metadata={'source': 'data/olympic.txt'}, page_content='올림픽은 국제경기연맹(IF), 국가 올림픽 위원회(NOC), 각 올림픽의 위원회(예-벤쿠버동계올림픽조직위원회)로 구성된다. 의사 결정 기구인 IOC는 올림픽 개최 도시를 선정하며, 각 올림픽 대회마다 열리는 올림픽 종목도 IOC에서 결정한다. 올림픽 경기 개최 도시는 경기 축하 의식이 올림픽 헌장에 부합하도록 조직하고 기금을 마련해야 한다. 올림픽 축하 행사로는 여러 의식과 상징을 들 수 있는데 올림픽기나 성화가 그 예이다.'),
 Document(metadata={'source': 'data/olympic.txt'}, page_content='위원회의 목표는 올림픽 종목에 더 체계적으로 다가가는 것이다. 위원

In [ ]:
#### invoke() 할 때 검색 조건 설정을 변경 - configuable(RunnableConfig)에 설정한다.
from langchain_core.runnables import ConfigurableField

retriever4 = vector_store.as_retriever(
    search_type="mmr",  #MMR 방식을 지정.
    search_kwargs = {
        "k":5,
        "fetch_k": 10,
        "lambda_mult":0.2, # 1<- 유사도를 강조, 0<- 다양성을 강조
        "filter":{"source":"data/olympic.txt"}
    }
)
retriever4 = retriever4.configurable_fields(
    # 파라미터 이름 = ConfigurableFieldSpec()->RunnableConfig의 어떤값으로 바꿀지 설정 
    search_kwargs=ConfigurableField(
        # search_kwargs의 인자 값을 RunnableConfig의 search_kwargs(id) key로 넘어오는 값으로 변경.
        id="search_kwargs",   
        name="Search Kwargs", # 설정이름 (생략가능)
        description="조회 파라미터값들을 변경" # 설명(생략가능)
    )
)

config = {
    "configurable":{
        "search_kwargs":{
            "k":2,
            "fetch_k": 10,
            "lambda_mult":0.7, # 1<- 유사도를 강조, 0<- 다양성을 강조
            "filter":{"source":"data/olympic.txt"}
        }
    }
}
retriever4.invoke(query, config=config)


[Document(metadata={'source': 'data/olympic.txt'}, page_content='국제 올림픽 위원회\n올림픽 활동이란 많은 수의 국가, 국제 경기 연맹과 협회 • 미디어 파트너를 맺기 • 선수, 직원, 심판, 모든 사람과 기관이 올림픽 헌장을 지키는 것을 말한다. 국제올림픽위원회(IOC)는 모든 올림픽 활동을 통솔하는 단체로서, 올림픽 개최 도시 선정, 계획 감독, 종목 변경, 스폰서 및 방송권 계약 체결 등의 권리가 있다. 올림픽 활동은 크게 세 가지로 구성된다.'),
 Document(metadata={'source': 'data/olympic.txt'}, page_content='위원회의 목표는 올림픽 종목에 더 체계적으로 다가가는 것이다. 위원회에서는 우선적으로 올림픽 종목으로 포함되기 위해서는 7개의 기준을 충족시켜야 한다고 말한다. 이 7개의 기준은 역사, 전통, 보편성, 인기도와 잠재성, 선수의 건강, 연맹의 스포츠를 관리할만한 능력, 스포츠를 여는 데에 필요한 비용이다. 예를 들면 2012년 하계 올림픽의 정식종목 후보에 7개 조건을 포함한 비(非)올림픽 스포츠가 올랐고 그 내용은, 골프, 가라테, 럭비, 인라인 스케이팅, 스쿼시였다. 이 스포츠들은 IOC 상임이사회에서 재검토되어 2005년 7월에 열린 싱가포르 총회에서')]

# TODO 다음을 작성한다.

In [34]:
from langchain_community.document_loaders import TextLoader
from langchain_chroma import Chroma
from langchain_openai import OpenAIEmbeddings, ChatOpenAI
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough
from langchain.globals import set_llm_cache
from langchain.cache import InMemoryCache, SQLiteCache
from langchain_core.output_parsers import StrOutputParser

set_llm_cache(SQLiteCache(".cache_prompt.sqlite"))

collection_name = "olmpic_docs"
persist_directory = "vector_store/chroma/olympic3"

# Text Loading
# Split
loader = TextLoader("data/olympic.txt", encoding="utf-8")
splitter = RecursiveCharacterTextSplitter.from_tiktoken_encoder(
    model_name="gpt-4o-mini", 
    chunk_size=1000,
    chunk_overlap=100
)
docs = loader.load_and_split(splitter)
print(len(docs))

# Vector Store 생성/연결
embedding_model = OpenAIEmbeddings(model="text-embedding-3-small")
vector_store = Chroma.from_documents(
    documents=docs,
    embedding=embedding_model, 
    collection_name=collection_name,
    persist_directory=persist_directory
)

# Retriever 생성 - "mmr" 방식
retriever = vector_store.as_retriever(
    search_type="mmr",
    search_kwargs={"k":5, "fetch_k":10}
)


18


In [37]:
# Prompt Template 생성
messages = [
        ("ai", """
You are a helpful assistanct. Answer question using only the following context. 
If you don't know the answer, just say you don't know. Don't make it up. 
Answer in Korean.

{context}"""),
        ("human", "{question}"),
]

prompt_template = ChatPromptTemplate(messages)

# 모델
model = ChatOpenAI(model="gpt-4o-mini")

# output parser
parser = StrOutputParser()

# Chain 구성 retriever(관련문서 조회) -> prompt_template(prompt생성) -> model(정답) -> output parser
chain = {"context":retriever, "question":RunnablePassthrough()} | prompt_template | model | parser

In [39]:
# Chain을 이용해 질의
query = "국제 올림픽 위원회에 대해서 설명해주세요."
query = "올림픽과 관련된 논란들을 정리해주세요."
result = chain.invoke(query)
print(result)

올림픽과 관련된 논란들은 다음과 같습니다:

1. **전쟁으로 인한 올림픽 취소**: 제1차 세계대전으로 인해 1916년 베를린 올림픽이 취소되었고, 제2차 세계대전 동안 1940년 도쿄 올림픽과 삿포로 동계 올림픽, 1944년 런던 올림픽과 코르티나담페초 동계 올림픽이 취소되었다.

2. **정치적 갈등**: 2008년 베이징 올림픽 개막식 날 조지아와 러시아 간의 전쟁이 발생하였으며, 이 사건은 올림픽의 정치적 맥락을 드러냈다.

3. **테러 사건**: 1972년 뮌헨 올림픽에서 이스라엘 선수 11명이 테러리스트에 의해 인질로 잡혀 사망한 사건과 1996년 애틀란타 올림픽에서 발생한 폭발 사건이 있었다.

4. **프로 선수 참가 문제**: 20세기 초, 프로 선수의 참가 불허로 인해 여러 분쟁이 있었으며, 특히 1912년 하계 올림픽에서 준프로야구선수로 활동한 짐 소프가 메달을 박탈당한 사례가 있다.

5. **아마추어 정신의 변화**: 아마추어 선수에 대한 정의가 시대에 뒤처지면서, 많은 국가들이 '정식 아마추어 선수'를 양성해 순수한 아마추어 정신을 벗어나고 있다는 비판이 제기되었다.

이와 같은 논란들은 올림픽의 역사와 발전 과정에서 중요한 이슈로 자리잡고 있다.


## Map Reduce 방식

- RAG에서 질문과 유사한 document들을 찾아  그대로 질문과 함께 전달하는 방식을 **stuff** 방식이라고 한다.
- Map reduce 방식은 document들에서 질문을 답하는데 적합한 document를 llm을 이용해 찾은 뒤 전달하는 방식이다.
  1. vectorstore에서 질문과 유사한 document들을 유사도 검색으로 찾음
     - 이것들은 단순 유사도라서 질문과 직접 관계 없는 것도 있다.
  2. document들이 질문을 답하는데 적합한지 llm에게 질문해서 관련있는 것들만 문자열로 묶어준다.
  3. 질문과 `2`에서 찾은 내용을 context로해서 llm에 질문하여 최종 답을 받는다.
- context를 좀더 질문과 관련된 내용으로 전달 할 수있는 장점이 있는 반면 비용이 드는 폐쇄형 llm을 사용하는 경우 비용이 추가로 드는 단점이 있다.

In [1]:
from langchain_chroma import Chroma
from langchain_openai import OpenAIEmbeddings, ChatOpenAI
from langchain.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough, RunnableLambda 
from langchain_core.output_parsers import StrOutputParser
from dotenv import load_dotenv
load_dotenv()

True

In [2]:
collection_name = "olmpic_docs"
persist_directory = "vector_store/chroma/olympic3"

# Vector DB와 연결. + retriever
embedding_model = OpenAIEmbeddings(model="text-embedding-3-small")
vector_store = Chroma(
    embedding_function=embedding_model,
    collection_name=collection_name,
    persist_directory=persist_directory
)
retriever = vector_store.as_retriever(search_kwargs={"k":10})

In [7]:
# vector store에서 찾은 문서들과 질문을 비교해서 진짜 관련 있는 문서들만 찾는 프롬프트.
## context: 찾은 문서, question: 질문
map_doc_prompt = ChatPromptTemplate.from_messages([
    ("system",  """
Use the following portion of a long document to see if any of the text is relevant to answer the question. 
Return any relevant text verbatim. If there is no relevant text, return : ''
-------
{context}
"""),
    ("human", "{question}"),
])

model = ChatOpenAI(model="gpt-4o-mini")

# 질문 - 문서 관련성을 비교하는 체인
map_doc_chain = map_doc_prompt | model
# map_doc_chain.invoke({"context":"사과는 맛있어요?", "question":"올림픽 종목에 대해 설명해줘."})
# map_doc_chain.invoke({"context":"올림픽에는 300개의 종목이 있습니다.", "question":"올림픽 종목에 대해 설명해줘."})

## retriever로 문서 조회 -> map_doc_chain으로 관련문서를 찾기 
def map_doc(inputs):
    """
    Runnable로 정의할 함수. 
    retriever가 조회한 문서들과 question을 받아서 map_doc_chain을 이용해 관련성을 확인한다.
    관련된 문서 내용만 모아서 반환.
    parameter
        inputs: dict[documents: list[Document], question:질문]. {"documents":retriever, "question":RunnablePassthrough()}
    """
    docs = inputs["documents"]   # list[Document, Document, Document, ...]
    question= inputs["question"] # str
    context = "" # 질문과 관련된 내용들만 모아 놓을 변수.
    for doc in docs:
        # Document와 question을 map_doc_chain에 전달해서 관련된 내용인지 확인.
        res = map_doc_chain.invoke({"context":doc.page_content, "question":question})
        context += res.content+"\n\n" # AIMessage.content

    return context

map_reduce_chain = {"documents":retriever, "question":RunnablePassthrough()} | RunnableLambda(map_doc)
# r = map_reduce_chain.invoke("올림픽 종목에 대해 설명해줘.")
# print(r)


In [9]:
final_prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            """
            Given the following extracted parts of a long document and a question, create a final answer. 
            If you don't know the answer, just say that you don't know. Don't try to make up an answer.
            ------
            {context}
            """,
        ),
        ("human", "{question}"),
    ]
)

chain = ({"context":map_reduce_chain, "question":RunnablePassthrough()} 
         | final_prompt
         | model
         | StrOutputParser())

In [10]:
result = chain.invoke("올림픽 경기 종목에 대해서 설명해줘. 목록형식으로 답변을 작성해줘.")
print(result)

올림픽 경기 종목에 대한 설명은 다음과 같습니다:

1. **하계 올림픽**
   - 총 33개 종목
   - 약 400개의 세부종목
   - 주요 종목 예시:
     - 육상
     - 수영
     - 펜싱
     - 체조
     - 레슬링 (자유형, 그레코로만형)
     - 사이클
     - 사격
     - 테니스
     - 역도
     - 조정

2. **동계 올림픽**
   - 총 7개 종목
   - 주요 종목 예시:
     - 크로스컨트리
     - 피겨 스케이팅
     - 아이스 하키
     - 노르딕 복합
     - 스키 점프
     - 스피드 스케이팅

3. **정식 종목 선정**
   - IOC의 승인을 받은 국제경기연맹의 관리 하에 있음
   - 정식종목으로 승인되기 위해서는 재적 위원 수의 과반수 이상 찬성 필요

4. **시범종목과 과거 종목**
   - 일부 종목은 처음에는 시범종목으로 시작하여 정식종목으로 승인됨 (예: 배드민턴, 농구, 배구)
   - 과거에 정식종목이었으나 현재는 빠진 종목도 존재 (예: 야구)

위의 정보는 올림픽 경기 종목에 대한 기본적인 설명입니다. 추가적인 정보가 필요하다면 특정 종목에 대해 질문해 주시면 더 자세히 설명해 드릴 수 있습니다.
